# Numerical results for RTS-96



In [1]:
####### IMPORTS ########
include("../src/tmp_inst_rts96.jl")
include("../src/TemporalInstanton.jl")
include("../src/TemporalInstanton2.jl")

using TemporalInstanton,TemporalInstanton2

####### LOAD DATA ########
psData = psDataLoad()

# unpack psDL (boilerplate):
(Sb,f,t,r,x,b,Y,bustype,
Gp,Gq,Dp,Dq,Rp,Rq,
Pmax,Pmin,Qmax,Qmin,Plim,
Vg,Vceiling,Vfloor,
busIdx,N,Nr,Ng,k) = unpack_psDL(psData)

####### LINK DATA ########
# Static
Ridx = find(Rp) # Vector of renewable nodes
Y = full(Y) # Full admittance matrix (ref not removed)
ref = 1 # Index of ref node
k = k # Conventional generator participation factors
tau = 1. # Heating constant
G0 = [Gp,Gp.*0.95,Gp.*0.9]
D0 = [Dp,Dp.*0.95,Dp.*0.9]
P0 = [Rp,Rp,Rp]
kQobj = 0 # Should always be zero; objective has no constant term
c = 1. # RHS of line temperature constraint
kQtheta = -c # Move to LHS so we have Q(z) = 0
lines = [(f[i],t[i]) for i in 1:length(f)];

n = size(Y,1)
nr = length(Ridx)
T = int(length(G0)/n) # infer number of time steps

# Create Qobj:
Qobj = tmp_inst_Qobj(n,nr,T)
# Augment Qobj with additional rows and columns of zeros:
Qobj = tmp_inst_pad_Q(full(Qobj),T)

# Create A1 (only A2 changes during opt.):
A1 = full(tmp_inst_A(Ridx,T,Y,ref,k))
A1 = [A1 zeros((n+1)*T,T)]

# Create b:
b = tmp_inst_b(n,T,G0,P0,D0)
# Augment b with new elements:
tmp_inst_pad_b(b,T)

# Create Qtheta:
Qtheta = tmp_inst_Qtheta(n,nr,T)#,tau)

# Form obj and constraint quadratics:
G_of_x = (Qobj,0,kQobj)
Q_of_x = (Qtheta,0,kQtheta)

(
279x279 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [2]:
score,x,θ,α,diffs = loop_through_lines(
    G_of_x,Q_of_x,A1,b,
    n,T,tau,Ridx,ref,lines[1:10])

([273.824,486.167,1860.69,2002.89,922.45,934.002,559.485,665.239,1157.96,1630.55,1631.77,1980.06],Array[[],[[-9.12835,0.362356,1.43256,1.57937,1.64775,0.981916,0.590053,1.71124,0.374529,0.0574299,0.170688,0.186063,0.206895,0.26214,0.292525,1.0429,1.05584,1.05068],[-8.7748,0.348322,1.37707,1.5182,1.58393,0.943885,0.567199,1.64496,0.360023,0.0552056,0.164077,0.178856,0.198882,0.251987,0.281195,1.00251,1.01495,1.00998],[-8.42124,0.334287,1.32159,1.45703,1.52011,0.905854,0.544346,1.57868,0.345517,0.0529812,0.157466,0.17165,0.190868,0.241834,0.269865,0.962113,0.974053,0.969288],[12.1632,-0.482829,-1.90884,-2.10446,-2.19557,-1.30837,-0.786228,-2.28018,-0.499048,-0.0765236,-0.227436,-0.247923,-0.275681,-0.349294,-0.389781,-1.38963,-1.40688,-1.39999],[11.6921,-0.464128,-1.83491,-2.02295,-2.11054,-1.2577,-0.755776,-2.19186,-0.479719,-0.0735597,-0.218627,-0.23832,-0.265004,-0.335765,-0.374684,-1.33581,-1.35239,-1.34577],[11.221,-0.445428,-1.76097,-1.94144,-2.0255,-1.20702,-0.725324,-2.10355,-0.4

In [5]:
Inf*ones(10)

10-element Array{Float64,1}:
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf

In [8]:
[x[2][1] x[2][4]]

18x2 Array{Float64,2}:
 -9.12835    12.1632   
  0.362356   -0.482829 
  1.43256    -1.90884  
  1.57937    -2.10446  
  1.64775    -2.19557  
  0.981916   -1.30837  
  0.590053   -0.786228 
  1.71124    -2.28018  
  0.374529   -0.499048 
  0.0574299  -0.0765236
  0.170688   -0.227436 
  0.186063   -0.247923 
  0.206895   -0.275681 
  0.26214    -0.349294 
  0.292525   -0.389781 
  1.0429     -1.38963  
  1.05584    -1.40688  
  1.05068    -1.39999  

In [11]:
α

10-element Array{Array{Float64,1},1}:
 []                                                     
 [-42.3335,-40.4741,-38.6147,-33.2915,-31.7823,-30.2731]
 [-37.2799,-35.6425,-34.005,-39.6782,-37.8943,-36.1105] 
 [-38.4108,-36.6974,-34.9839,-38.5034,-36.7983,-35.0932]
 [-39.5615,-37.7891,-36.0168,-37.2525,-35.612,-33.9715] 
 []                                                     
 []                                                     
 [-38.3978,-36.6912,-34.9847,-38.5272,-36.8146,-35.1021]
 [-39.5421,-37.7833,-36.0246,-37.2616,-35.6068,-33.952] 
 []                                                     

In [9]:
score

12-element Array{Float64,1}:
  273.824
  486.167
 1860.69 
 2002.89 
  922.45 
  934.002
  559.485
  665.239
 1157.96 
 1630.55 
 1631.77 
 1980.06 

In [21]:
function loop_through_lines(
    G_of_x,
    Q_of_x,
    A1,
    b,
    n,
    T,
    tau,
    Ridx,
    ref,
    lines)
    
    # Initialize vars used to store results:
    score = Float64[]
    α = Array(Vector{Float64},0)

    θ = Array(Array,0)
    x = Array(Array,0)
    diffs = Array(Array,0)

    # Loop through all lines:
    for line in lines
        # array of vec. with Float values:
        deviations = Array(Vector{Float64},0)
        angles = Array(Vector{Float64},0)
        alpha = Float64[]

        # Create A2 based on chosen line:
        A2 = tmp_inst_A_scale(n,Ridx,T,tau,ref,line)
        # Stack A1 and A2:
        A = [A1; A2]

        opt,solutions = solve_temporal_instanton(G_of_x,Q_of_x,A,b,T)

        for xvec in opt
            push!(score,(xvec'*(G_of_x[1])*xvec)[1])
        end

        # Variable breakdown:
        # (nr+n+1) per time step
        #   First nr are deviations
        #   Next n are angles
        #   Last is mismatch
        # T variables at the end: anglediffs
        for xvec in opt
            for t = 1:T
                push!(deviations,xvec[(nr+n+1)*(t-1)+1:(nr+n+1)*(t-1)+nr])
                push!(angles,xvec[(nr+n+1)*(t-1)+nr+1:(nr+n+1)*(t-1)+nr+n])
                push!(alpha,xvec[(nr+n+1)*(t)])
                push!(diffs,xvec[end-T+1:end])
            end
        end
        push!(x,deviations)
        push!(θ,angles)
        push!(α,alpha)
    end
    return score,x,θ,α,diffs
end

loop_through_lines (generic function with 1 method)

In [3]:
function solve_temporal_instanton(G_of_x,Q_of_x,A,b,T)
    
    Qobj = G_of_x[1]
    c = - Q_of_x[3]
    
    opt = Array(Vector{Float64},0)
    
    # Partition A:
    A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)

    # Find translation point:
    x_star = find_x_star(Qobj,A,b,T)

    # Translate quadratics:
    G_of_y = translate_quadratic(G_of_x,x_star)
    Q_of_y = translate_quadratic(Q_of_x,x_star)

    N = kernel_rotation(A)[:,1:size(A,2) - rank(A)] # take only first k cols

    N1,N2,N3 = N[idx1,:],N[idx2,:],N[idx3,:] # partition N

    G_of_z = rotate_quadratic(G_of_y,N')
    Q_of_z = rotate_quadratic(Q_of_y,N')

    D,U = eig(Q_of_z[1])
    D = round(D,10)

    K = return_K(D)

    G_of_w = rotate_quadratic(G_of_z,(U/K)')
    Q_of_w = rotate_quadratic(Q_of_z,(U/K)')

    B11,B12,B21,B22,b1,b2 = partition_B(G_of_w,Q_of_w)

    Bhat,bhat = return_Bhat(B11,B12,B22,b1,b2)

    eps = 1e-8
    w0 = find_w(0,Bhat,bhat/2)

    if abs((w0'*w0) - c)[1] < eps
        println("v=0 works!")
    end

    solutions, vectors = solve_secular(Bhat,bhat/2,-Q_of_w[3])

    for vec in vectors
        push!(opt,return_xopt(vec,B11,B12,b1,N,U,K,x_star))
    end
    
    return opt,solutions
end

solve_temporal_instanton (generic function with 1 method)